In [1]:
# Politico, state by state
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import time
import os
import numpy as np
import pandas as pd
import pandas as pd
import json
import traceback

import sys
print(sys.version)

3.4.5 |Anaconda 4.1.1 (64-bit)| (default, Jul  5 2016, 14:53:07) [MSC v.1600 64 bit (AMD64)]


In [2]:
# Choose what kind of Selenium webdriver to use. This one uses Chrome webdriver; for this to
# work, the Chrome webdriver executable has to be in the system PATH
def make_webdriver(kind='Chrome'):
    if kind == 'Chrome':
        return webdriver.Chrome()
    elif kind == 'nonexistent': # you can put your own here
        #return webdriver.whatevergoeshere
        raise Exception # remove this
    else:
        raise Exception

In [3]:
import logging

log_name = 'politico'
log_dir = ''

logger = logging.getLogger(log_name)
log_handler = logging.FileHandler(os.path.join(log_dir, log_name + '.log'))
log_formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
log_handler.setFormatter(log_formatter)
if logger.handlers == []:
    logger.addHandler(log_handler) 
    logger.setLevel(logging.INFO)
logger.info(log_name + 'session started')

In [4]:
use_jupyter_pdb = True
if use_jupyter_pdb:
    from IPython.core.debugger import Tracer
# to activate, use Tracer()()

In [5]:
states = """Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
Nevada
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming""".split('\n')

states = [x.strip() for x in states]



In [55]:
%%time
if os.path.isfile('results.json'):
    results = json.load(open('results.json', 'r'))
else:
    results = {}

if os.path.isfile('metadata.json'):
    metadata = json.load(open('metadata.json', 'r'))
else:
    metadata = {}

Wall time: 914 ms


In [7]:
section2urlformat = {'Presidential General Election': 'http://www.politico.com/2016-election/results/map/president/{}/',
                     'Presidential Primaries': 'http://www.politico.com/2016-election/primary/results/map/president/{}/',
                     'Senate General Election': 'http://www.politico.com/2016-election/results/map/senate/{}/',
                     'Senate Primaries': 'http://www.politico.com/2016-election/primary/results/map/senate/{}/',
                     'House General Election': 'http://www.politico.com/2016-election/results/map/house/{}/',
                     'House Primaries': 'http://www.politico.com/2016-election/primary/results/map/house/{}/',
                     'Governor General Election': 'http://www.politico.com/2016-election/results/map/governor/{}/',
                     'Governor Primaries': 'http://www.politico.com/2016-election/primary/results/map/governor/{}/',
                     'Ballot Measures': 'http://www.politico.com/2016-election/results/map/ballot-measures/{}/'
                     }

section_names = sorted(list(section2urlformat.keys()))

print(section_names)


['Ballot Measures', 'Governor General Election', 'Governor Primaries', 'House General Election', 'House Primaries', 'Presidential General Election', 'Presidential Primaries', 'Senate General Election', 'Senate Primaries']


In [118]:
# use selenium to scroll down to the bottom of the page so that all html is loaded, then process with BeautifulSoup

# to limit the sections or states done in this block (e.g. to fix errors), redefine these variables:
section_names_to_do = None
states_to_do = None
logging_verbose = False
log_errors = False # If False, throws exception

nondefault_items = False

if section_names_to_do is None:
    section_names_to_do = section_names
else:
    nondefault_items = True
if states_to_do is None:
    states_to_do = states
else:
    nondefault_items = True
    
if nondefault_items and logging_verbose:
    logger.info('Verbose logging activated for sections {} and states {}'.format(', '.join(section_names_to_do),
                                                                                     ', '.join(states_to_do)))
results_changed = False
metadata_changed = False

for section in section_names_to_do:
    print('')
    print(section, end=' ')
    logger.info('SECTION {}'.format(section))
    driver = webdriver.Chrome()
    if section not in metadata.keys():
        metadata[section] = {'states_done': [],
                             'states_not_applicable': [],
                             'states_error': {},
                             'states_fips': {}}
        logger.info('metadata blank created')
        metadata_changed = True
    if section not in results.keys():
        logger.info('results blank created')
        results[section] = []  
        results_changed = True
    if logging_verbose:
        logger.info('*VERBOSE: section '+section)
    states_to_ignore = [x for x in states_to_do if x in metadata[section]['states_done'] or x 
                        in metadata[section]['states_not_applicable']]
    for state in states_to_do:
        if state in states_to_ignore:
            if logging_verbose:
                if state in metadata[section]['states_done']:
                    logger.info('*VERBOSE: state {} ignored because done'.format(state))
                else:
                    logger.info('*VERBOSE: state {} ignored because not_applicable'.format(state))
            continue
        print(state, end=' ')
        this_results = []
        this_log = "    %line = {{'section': '{}', 'state': '{}', ".format(section,state)
        try:
            state_url = state.lower().replace(' ', '-')
            # make sure this section is in the results
            driver.get('http://www.politico.com/2016-election/results/{}/'.format(state_url))
            time.sleep(10)
            outerhtml = driver.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(outerhtml, 'lxml')
            found = False
            #Tracer()()
            for tag in soup.find_all('a'):
                if tag.text == section:
                    found = True
                if tag.text == section.replace('Primaries', 'Caucuses'):
                    found = True
                if tag.text == section.replace('Primaries', 'Primary'):
                    found = True
                if tag.text == section.replace('Primaries', 'Caucus'):
                    found = True
                if tag.text == section.replace('Primaries', 'Primary and Caucus'):
                    found = True
                if tag.text == section.replace('Primaries', 'Caucus and Primary'):
                    found = True
                if tag.text == section.replace('Primaries', 'Race'):
                    found = True # for Washington's open primary
                if tag.text == section.replace('Primaries', 'Races'):
                    found = True
            if not found:                 
                metadata[section]['states_not_applicable'].append(state)
                logger.info(this_log + "'status': 'not_applicable'}}")
                continue
            # if section found, scrape
            url = section2urlformat[section].format(state_url)
            driver.get(url)
            time.sleep(10) # let it load; very important. There are more robust ways to ensure the
                           # page is loaded, but I was lazy.
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            outerhtml = driver.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(outerhtml, 'lxml')
            if section == 'Ballot Measures':
                is_bm = True
                name_type = 'ballot_measure_type'
            else:
                name_type = 'geo_name'
                is_bm = False
            for n_article, article in enumerate(soup.find_all('article', attrs={'class': 'results-group'})):
                fips = article.attrs['data-fips']
                header_name = article.find_all('div', attrs={'class': 'title'})[0].text.strip()
                if state == 'Iowa': # really unprofessional quick fix because I'm tired...
                    header_name.replace('O&apos;Brien', "O'Brien")
                reporting_pct = None
                if section == 'Presidential General Election':
                    if fips == '0':
                        text = article.find_all('p', attrs={'class': 'reporting'})[0].text
                        electoral_votes = int(re.search('Electoral Votes: (\d+)', text).group(1))
                    else:
                        electoral_votes = None
                # figure out if divided into parties, such as primaries/caucuses
                subdivisions = article.find_all('div', attrs={'class': 'results-data'})
                if 'pos-omega' in subdivisions[0].attrs['class']:
                    subdivisions = [article]
                    has_subdivisions = False
                    party = None
                    reporting_pct = float(re.search('(^.+)%', article.find_all('p',
                                                attrs={'class': 'reporting'})[0].text).group(1))
                else:
                    has_subdivisions = True
                for subdiv in subdivisions:
                    skip = False
                    if has_subdivisions:
                        if (section=="House Primaries" and state=='Illinois' and 
                              (header_name.find('5th') != -1 or
                               header_name.find('12th') != -1)):
                            skip = True
                        else:
                            text = subdiv.find_all('div', attrs={'class': 'title'})[0].text.strip()
                            found = re.search('(\D+)([\.\d]+)% Reporting', text)
                            # Politico shows "no county level results will be reported for"
                            # the following primaries/caucuses:
                            if ((not found and state=='Colorado' and text=='Republican') or
                                (not found and state=='Wyoming' and text=='Republican') or
                                (not found and state=='Maine' and text=='Republican')):
                                skip = True
                            else: 
                                party = found.group(1).strip()
                                reporting_pct = float(found.group(2))                    
                    if not skip and not is_bm:
                        table = subdiv.find_all('table', attrs={'class': 'results-table'})[0]
                        has_winner = False
                        winners = subdiv.find_all('span', attrs={'class': 'token token-winner'})
                        if len(winners) > 0:
                            has_winner = True
                        for i, tr in enumerate(table.find_all('tr')):
                            row = {'state': state, name_type: header_name, 'fips': fips, 'rank': i+1,
                                   'party': party, 'reporting_pct': reporting_pct}
                            if section == 'Presidential General Election':
                                row['electoral_votes'] = electoral_votes
                            abbrs = tr.find_all('abbr')
                            if len(abbrs) > 0:
                                individual_party = tr.find_all('abbr')[0].text
                                assert len(individual_party) > 0
                                row['individual_party'] = individual_party
                            if has_winner:
                                winner = tr.find_all('span', attrs={'class': 'token token-winner'})
                                if len(winner) > 0:
                                    is_winner = True
                                else:
                                    is_winner = False
                                row['is_winner'] = is_winner
                            name = str(tr.find_all('span', attrs={'class': 'name-combo'})[0].contents[-1]).strip()
                            assert len(name) > 0
                            found = re.search('(^.+ )\([iI]\)$', name.strip())
                            if found:
                                row['name'] = found.group(1).strip()
                                row['is_incumbent'] = True
                            else:
                                row['name'] = name.strip()
                                row['is_incumbent'] = False
                            vote_pct = tr.find_all('td', attrs={'class': 'results-percentage'})
                            if len(vote_pct) == 0:
                                row['vote_pct'] = None
                            else:
                                vote_pct = vote_pct[0].text
                                if len(vote_pct) == 0:
                                    row['vote_pct'] = None
                                else:
                                    vote_pct = vote_pct.replace('%', '')
                                    try:
                                        row['vote_pct'] = float(vote_pct)
                                    except ValueError:
                                        row['vote_pct'] = vote_pct # string e,g, 'Uncontested
                            votes = tr.find_all('td', attrs={'class': 'results-popular'})
                            if len(votes) == 0:
                                row['votes'] = None
                            else:
                                votes = votes[0].text
                                if len(votes) == 0:
                                    row['votes'] = None
                                else:
                                    votes = votes.replace(',', '')
                                    try:
                                        row['votes'] = int(votes)
                                    except ValueError:
                                        row['votes'] = votes # string e,g, 'Uncontested'
                            delegates = tr.find_all('td', attrs={'class': 'delegates-cell'})
                            if len(delegates) > 0:
                                delegates = delegates[0].text
                                if len(delegates) > 0:
                                    delegates = int(delegates.replace(',', ''))
                                    row['delegates'] = delegates
                            this_results.append(row)
                    elif not skip: # ballot measure
                        row = {}
                        for detail in subdiv.find_all('div', attrs={'class': 'results-detail'}):
                            title = detail.find_all('h6')[0].text.strip()
                            summary = detail.find_all('p', attrs={'class': 'summary'})[0].text.strip()
                            table = detail.find_all('table', attrs='results-table')[0]
                            for tr in table.find_all('tr', attrs='type-ballot'):
                                row = {'state': state, name_type: header_name, 'fips': fips,
                                      'reporting_pct': reporting_pct, 'title': title, 'summary': summary}
                                text = tr.find_all('span', attrs={'class': 'name-combo'})[0].text
                                if 'Winner' in text:
                                    row['is_winner'] = True
                                    row['choice'] = text.replace('Winner', '').strip()
                                else:
                                    row['is_winner'] = False
                                    row['choice'] = text.strip()
                                row['vote_pct'] = float(tr.find_all('span', attrs={'class': 'number'})[0].text.replace('%', ''))
                                row['votes'] = int(tr.find_all('td', attrs={'class': 'results-popular'})[0].text.replace(',', ''))
                                this_results.append(row)
                                
                    for row in this_results:
                        results[section].append(row)
                        results_changed = True

                    if state not in metadata[section]['states_fips'].keys():
                        metadata[section]['states_fips'][state] = [fips]
                    else:
                        metadata[section]['states_fips'][state].append(fips)
                    metadata_changed = True
            logger.info(this_log+ "'status': 'done', 'fips': {}, 'row': {}}}".format(n_article+1, repr(row)))
            metadata[section]['states_done'].append(state)
            metadata_changed = True
        except:
            print('<-err', end=' ')
            err_type, err_val, sys_tb_object = sys.exc_info()
            error_message = [str(err_type).replace('<class \'', '').replace('\'>', ''), err_val.__doc__]
            for i, tbtext in enumerate(traceback.format_tb(sys_tb_object)):
                toret = tbtext.split('\n')
                error_message.append(str(i))
                error_message.append(toret[0].strip())
                error_message.append(toret[1].strip())
            error_message = ' | '.join(error_message)
            if log_errors:
                logger.info(this_log+"'status': error', 'error_message': {}".format(repr(error_message)))
                metadata[section]['states_error'][state] = error_message
                metadata_changed = True
            else:
                print(error_message)
                raise Exception
    driver.quit()
    
if results_changed:
    with open('results.json', 'w+') as f:
        f.write(json.dumps(results))
if metadata_changed:
    with open('metadata.json', 'w+') as f:
        f.write(json.dumps(metadata))
        
if logging_verbose:
    logging.verbose = False
    logger.info('Verbose logging deactivated')
    
# create dataframes from results, drop duplicate results
dfs = {}
for section in section_names:
    dfs[section] = pd.DataFrame(results[section]).drop_duplicates()
    results[section] = dfs[section].to_dict(orient='records')
with open('results.json', 'w+') as f:
    f.write(json.dumps(results))


Ballot Measures 
Governor General Election 
Governor Primaries 
House General Election 
House Primaries 
Presidential General Election 
Presidential Primaries 
Senate General Election 
Senate Primaries 

In [89]:
# Check consistencies in geographical names in Presidential Primaries vs Presidential General Election:
df1 = dfs['Presidential Primaries']
df2 = dfs['Presidential General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([x for x in counties1 if x.find('Overall') == -1])
    df2_ = df2[df2.state == state]
    counties2 = list(df2_.geo_name.unique())
    counties2 = set([x for x in counties2 if x.find('Overall') == -1])
    if counties1 != counties2:
        if state == 'Kansas':
            print('\nKansas primary results missing from Politico')
        elif state == 'North Dakota':
            print('\nNorth Dakota primary results missing from Politico')
        elif state == 'Minnesota':
            print("\nMinnesota's presidential primaries are by congressional district, vote is by county")
        else:
            print('')
            print(state)
            diff1 = counties1.difference(counties2)
            diff2 = counties2.difference(counties1)
            if len(diff1) > 0:
                print('Primary only: '+', '.join(sorted(list(diff1))))
            if len(diff2) > 0:
                print('General only: '+', '.join(sorted(list(diff2))))
            try:
                last1 = sorted(list(counties1))[-1]
            except:
                last1 = 'None'
            try:
                last2 = sorted(list(counties2))[-1]
            except:
                last2 = 'NONE'
            if last1 != last2:
                print('Last in primaries: '+last1)
                print('Last in general: '+last2)



Kansas primary results missing from Politico

Minnesota's presidential primaries are by congressional district, vote is by county

North Dakota primary results missing from Politico

South Dakota
Primary only: Shannon County
General only: Oglala Lakota County


In [90]:
# Check consistencies in geographical names in Senate Primaries vs Senate General Election:

def fix_name(text):
    return text.replace('Primary', '').replace('Primaries', '').replace('Election', '').strip()

df1 = dfs['Senate Primaries']
df2 = dfs['Senate General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([x for x in counties1 if x.find('Overall') == -1])
    df2_ = df2[df2.state == state]
    counties2 = list(df2_.geo_name.unique())
    counties2 = set([x for x in counties2 if x.find('Overall') == -1])
    if counties1 != counties2:
        if state == 'Louisiana':
            print('No page for Senate primaries exists on Politico for Louisiana')
        else:
            print('')
            print(state)
            diff1 = counties1.difference(counties2)
            diff2 = counties2.difference(counties1)
            if len(diff1) > 0:
                print('Primary only: '+', '.join(sorted(list(diff1))))
            if len(diff2) > 0:
                print('General only: '+', '.join(sorted(list(diff2))))
            try:
                last1 = sorted(list(counties1))[-1]
            except:
                last1 = 'None'
            try:
                last2 = sorted(list(counties2))[-1]
            except:
                last2 = 'NONE'
            if last1 != last2:
                print('Last in primaries: '+last1)
                print('Last in general: '+last2)


No page for Senate primaries exists on Politico for Louisiana

South Dakota
Primary only: Shannon County
General only: Oglala Lakota County


In [91]:
# Check consistencies in geographical names in House Primaries vs General Election:

def fix_name(text):
    return text.replace('Primary', '').replace('Primaries', '').replace('Election', '').strip()

df1 = dfs['House Primaries']
df2 = dfs['House General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([fix_name(x) for x in counties1 if x.find('Overall') == -1])
    df2_ = df2[df2.state == state]
    counties2 = list(df2_.geo_name.unique())
    counties2 = set([fix_name(x) for x in counties2 if x.find('Overall') == -1])
    if counties1 != counties2:
        if state == 'Louisiana':
            print('No page for House primaries exists on Politico for Louisiana')
        elif state == 'Virginia':
            print('Politico only has House primary results for 2nd, 4th and 6th districts in Virginia')
        else:
            print('')
            print(state)
            diff1 = counties1.difference(counties2)
            diff2 = counties2.difference(counties1)
            if len(diff1) > 0:
                print('Primary only: '+', '.join(sorted(list(diff1))))
            if len(diff2) > 0:
                print('General only: '+', '.join(sorted(list(diff2))))
            try:
                last1 = sorted(list(counties1))[-1]
            except:
                last1 = 'None'
            try:
                last2 = sorted(list(counties2))[-1]
            except:
                last2 = 'NONE'
            if last1 != last2:
                print('Last in primaries: '+last1)
                print('Last in general: '+last2)


No page for House primaries exists on Politico for Louisiana
Politico only has House primary results for 2nd, 4th and 6th districts in Virginia


In [92]:
# Check consistencies in geographical names in Governor Primaries vs General Election:
df1 = dfs['Governor Primaries']
df2 = dfs['Governor General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([x for x in counties1 if x.find('Overall') == -1])
    df2_ = df2[df2.state == state]
    counties2 = list(df2_.geo_name.unique())
    counties2 = set([x for x in counties2 if x.find('Overall') == -1])
    if counties1 != counties2:
        if False:
            pass
        else:
            print('')
            print(state)
            diff1 = counties1.difference(counties2)
            diff2 = counties2.difference(counties1)
            if len(diff1) > 0:
                print('Primary only: '+', '.join(sorted(list(diff1))))
            if len(diff2) > 0:
                print('General only: '+', '.join(sorted(list(diff2))))
            try:
                last1 = sorted(list(counties1))[-1]
            except:
                last1 = 'None'
            try:
                last2 = sorted(list(counties2))[-1]
            except:
                last2 = 'NONE'
            if last1 != last2:
                print('Last in primaries: '+last1)
                print('Last in general: '+last2)


In [95]:
# cross check presidential and senate general election (for states that had a senate election):
df1 = dfs['Senate General Election']
df2 = dfs['Presidential General Election']
for state in states:
    df1_ = df1[df1.state == state]
    counties1 = list(df1_.geo_name.unique())
    counties1 = set([x for x in counties1 if x.find('Overall') == -1])
    if len(counties1) > 0:
        df2_ = df2[df2.state == state]
        counties2 = list(df2_.geo_name.unique())
        counties2 = set([x for x in counties2 if x.find('Overall') == -1])
        if counties1 != counties2:
            if False:
                pass
            else:
                print('')
                print(state)
                diff1 = counties1.difference(counties2)
                diff2 = counties2.difference(counties1)
                if len(diff1) > 0:
                    print('Senate only: '+', '.join(sorted(list(diff1))))
                if len(diff2) > 0:
                    print('Presidential only: '+', '.join(sorted(list(diff2))))
                try:
                    last1 = sorted(list(counties1))[-1]
                except:
                    last1 = 'None'
                try:
                    last2 = sorted(list(counties2))[-1]
                except:
                    last2 = 'NONE'
                if last1 != last2:
                    print('Last in Senate: '+last1)
                    print('Last in Presidential: '+last2)


In [14]:
# to redo a section due to a data issue, use this function to remove it from the metadata

def remove_metadata(section, category, state):
    global metadata
    if type(metadata[section][category]) == dict:
        assert state in metadata[section][category].keys()
        metadata[section][category] = {k: v for k, v in metadata[section][category].items() if k != state}
    else:
        assert state in metadata[section][category]
        metadata[section][category].remove(state)
    logger.info('REMOVED: {} | {} | {}'.format(section, category, state))
        

In [97]:
for section in section_names:
    for k, v in metadata[section].items():
        print(section, k, len(v))

Ballot Measures states_fips 35
Ballot Measures states_error 0
Ballot Measures states_done 51
Ballot Measures states_not_applicable 0
Governor General Election states_fips 12
Governor General Election states_error 0
Governor General Election states_done 12
Governor General Election states_not_applicable 39
Governor Primaries states_fips 12
Governor Primaries states_error 0
Governor Primaries states_done 12
Governor Primaries states_not_applicable 39
House General Election states_fips 50
House General Election states_error 0
House General Election states_done 50
House General Election states_not_applicable 1
House Primaries states_fips 49
House Primaries states_error 1
House Primaries states_done 49
House Primaries states_not_applicable 2
Presidential General Election states_fips 51
Presidential General Election states_error 0
Presidential General Election states_done 51
Presidential General Election states_not_applicable 0
Presidential Primaries states_fips 51
Presidential Primaries sta

In [99]:
# Other inconsistencies

for state in states:
    errors = []
    
    for section in section_names:
        found_in = []
        if state in metadata[section]['states_error'].keys():
            errors.append('ERROR '+section+' '+metadata[section]['states_error'][state])
            found_in.append('errors')
        if state in metadata[section]['states_done']:
            found_in.append('done')
        if state in metadata[section]['states_not_applicable']:
            found_in.append('not_applicable')
        if len(found_in) == 0:
            errors.append('not found in {} metadata'.format(section))
        elif len(found_in) > 1:
            errors.append('found in {} metadata in mutually exclusive categories: '.format(section,
                                ','.join(found_in)))
        if (state in metadata[section]['states_fips'].keys() and ('done' not in found_in and 'errors'
                not in found_in)):
            errors.append('found in {} states_fips but not in states_done or states_errors'.format(section))
        
    if (state in metadata['Ballot Measures']['states_done'] and
        len( metadata['Ballot Measures']['states_fips']) != 2):
        '{} Ballot Measures tables'.format(len( metadata['Ballot Measures']['states_fips']))
    if (state in metadata['Governor General Election']['states_done'] and not
        state in metadata['Governor Primaries']['states_done']):
        errors.append('Governor election but no primary')
        if remove_from_primary_na:
            remove_metadata('Governor Primaries', 'states_not_applicable', state)
    if (state in metadata['Governor Primaries']['states_done'] and not
        state in metadata['Governor General Election']['states_done']):
        errors.append('Governor primary but no election')
    if (state in metadata['House General Election']['states_done'] and not
        state in metadata['House Primaries']['states_done']):
        errors.append('House election but no primary')
        if remove_from_primary_na:
            remove_metadata('House Primaries', 'states_not_applicable', state)
    if (state in metadata['House Primaries']['states_done'] and not
        state in metadata['House General Election']['states_done']):
        errors.append('House primary but no election')
    if (state in metadata['Senate General Election']['states_done'] and not
        state in metadata['Senate Primaries']['states_done']):
        errors.append('Senate election but no primary')
        if remove_from_primary_na:
            remove_metadata('Senate Primaries', 'states_not_applicable', state)
    if (state in metadata['Senate Primaries']['states_done'] and not
        state in metadata['Senate General Election']['states_done']):
        errors.append('Senate primary but no election')
    if (state in metadata['Presidential General Election']['states_done'] and not
        state in metadata['Presidential Primaries']['states_done']):
        if remove_from_primary_na:
            remove_metadata('Presidential Primaries', 'states_not_applicable', state)
        errors.append('Presidential election but no primary')
    if (state in metadata['Presidential Primaries']['states_done'] and not
        state in metadata['Presidential General Election']['states_done']):
        errors.append('Presidential primary but no election')
    try:
        n1 = len(metadata['Presidential General Election']['states_fips'][state])
        n2 = len(metadata['Senate General Election']['states_fips'][state])
        if n1 != n2:
            errors.append('{} tables in Presidential General Election, {} in Senate'.format(n1, n2))
    except KeyError:
        pass
    try:
        n1 = len(metadata['Presidential Primaries']['states_fips'][state])
        n2 = len(metadata['Senate Primaries']['states_fips'][state])
        if n1 != n2:
            if state not in ['Alaska']: #manual
                errors.append('{} tables in Presidential Primaries, {} in Senate'.format(n1, n2))
    except KeyError:
        pass
    if len(errors) > 0:
        print('')
        print(state)
        print('-'*len(state))
        for error in errors:
            print(error)


Colorado
--------
155 tables in Presidential Primaries, 260 in Senate

Florida
-------
136 tables in Presidential Primaries, 204 in Senate

Hawaii
------
10 tables in Presidential Primaries, 25 in Senate

Kansas
------
2 tables in Presidential Primaries, 212 in Senate

Louisiana
---------
House election but no primary
Senate election but no primary

Missouri
--------
252 tables in Presidential Primaries, 464 in Senate

New York
--------
126 tables in Presidential Primaries, 189 in Senate

North Dakota
------------
1 tables in Presidential Primaries, 162 in Senate

Oklahoma
--------
156 tables in Presidential Primaries, 234 in Senate

Utah
----
60 tables in Presidential Primaries, 120 in Senate

Washington
----------
80 tables in Presidential Primaries, 40 in Senate

Wisconsin
---------
146 tables in Presidential Primaries, 219 in Senate


In [102]:
# all of the above inconsistencies have been investigated, and they are
# not misrepresentations of the data as reported by Politico.

In [116]:
def remove_unvarying_columns(df):
    for col in df.columns:
        if len(df[col].unique()) == 1:
            df = df.drop(col, axis=1)
    return df

In [117]:
# save individual csvs
df = remove_unvarying_columns(dfs['Ballot Measures'])
df.to_csv('ballot_measures_2016.csv', index=False)
for section in ['Presidential General Election',
 'Presidential Primaries',
 'Senate General Election',
 'Senate Primaries',
 'Governor General Election',
 'Governor Primaries']:
    df = dfs[section]
    df1 = df[df.geo_name.str.contains('Overall')]
    df1 = remove_unvarying_columns(df1)
    assert len(df1) > 0
    df2 = df[~(df.geo_name.str.contains('Overall'))]
    df2 = remove_unvarying_columns(df2)
    name = section.lower().replace(' ', '_') + '_2016'
    df1.to_csv(name+'.csv', index=False)
    df2.to_csv(name+'_by_county.csv', index=False)
for section in ['House General Election', 'House Primaries']:
    df = remove_unvarying_columns(dfs[section])
    name = section.lower().replace(' ', '_') + '_2016'
    df.to_csv(name+'.csv', index=False)
    